# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging 
import os
import csv

from matplotlib import pyplot as plt 
import numpy as np
import pandas as pd

import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

print("SDK Version:",azureml.core.VERSION)

SDK Version: 1.18.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
found = False

key = "house_sale"
description_text = "house sales prediction dataset"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:

    data = 'https://www.kaggle.com/harlfoxem/housesalesprediction/download'
    dataset = Dataset.Tabular.from_delimited_files(data)
    dataset = dataset.register(workspace=ws,name = key,description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Avinash-AutoML1'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aml_compute_cluster = "Avinash-Test"

try:
    compute_target = ComputeTarget(workspace=ws,name = aml_compute_cluster)
    print('Found Exisitng cluster, use it ')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',max_nodes = 4,)
    compute_target = ComputeTarget.create(ws,aml_compute_cluster)


compute_target.wait_for_completion(show_output=True)

Found Exisitng cluster, use it 
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
found = False

key = "house_sales"
description_text = "house sales prediction dataset"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(df,test_size = 0.2,random_state = 42)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping": True,
    "primary_metric" : 'normalized_mean_absolute_error',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task = 'regression',
    debug_log = 'automl_reg_errors.log',
    training_data = x_train,
    label_column_name = "price",
    **automl_settings
)

In [7]:
from azureml.widgets import RunDetails
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

RunDetails(remote_run).show()
remote_run.get_status()
remote_run.wait_for_completion()


Running on local machine
Parent Run ID: AutoML_b99bfd84-b37f-47a3-8025-d54cf52d073c

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardi

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_b99bfd84-b37f-47a3-8025-d54cf52d073c',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-12-04T18:28:08.677844Z',
 'endTimeUtc': '2020-12-04T18:48:44.748108Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"Avinash-AutoML1","subscription_id":"7395406a-64a8-4774-b0c2-0d5dafb2a8ce","resource_group":"aml-quickstarts-129197","workspace_name":"quick-starts-ws-129197","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"normalized_mean_absolute_error","task_type":"regression","data_script":null,"validation_size":0.0

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run,fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model.steps)


Run(Experiment: Avinash-AutoML1,
Id: AutoML_b99bfd84-b37f-47a3-8025-d54cf52d073c_0,
Type: None,
Status: Completed)
[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                feature_sweeping_config=None, feature_sweeping_timeout=None,
                featurization_config=None, force_text_dnn=None,
                is_cross_validation=None, is_onnx_compatible=None, logger=None,
                observer=None, task=None, working_dir=None)), ('MaxAbsScaler', MaxAbsScaler(copy=True)), ('LightGBMRegressor', LightGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                  importance_type='split', learning_rate=0.1, max_depth=-1,
                  min_child_samples=20, min_child_weight=0.001,
                  min_split_gain=0.0, n_estimators=100, n_jobs=1, num_leaves=31,
                  objective=None, random_state=None, reg_alpha=0.0,
                  reg_lambda=0.0, silent=True, subsample=1.0,
                  sub

In [9]:
#TODO: Save the best model
import joblib

joblib.dump(fitted_model,"housing_sales.pkl")


['housing_sales.pkl']

In [16]:
model = best_run.register_model(model_path = './outputs',model_name = 'house_sale')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")


inference_config = InferenceConfig(entry_script="scoring.py",environment=myenv)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    auth_enabled=True,
    memory_gb=1,
    tags = {'name':'housing sales'},
    description='housing sales model'
)

service = Model.deploy(workspace=ws,
                        name = 'house-sales',
                        models= [model],
                        inference_config=inference_config,
                        deployment_config=aci_config
)

service.wait_for_deployment(True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [36]:
#y_test = x_test.pop("price")
import json 

x_test = x_test.to_numpy()

test_s = x_test[:5].tolist()
print(test_s)
test_samples = json.dumps({"data":test_s})
test_samples = bytes(test_samples,encoding = 'utf8')
print(test_samples)

result = service.run(input_data = test_samples)
print(result)

In [27]:
service.get_logs()

'2020-12-04T20:12:34.9401333Z stdout F 2020-12-04T20:12:34,781881900+00:00 - iot-server/run \n2020-12-04T20:12:34.9421172Z stdout F 2020-12-04T20:12:34,784496400+00:00 - rsyslog/run \n2020-12-04T20:12:34.9661945Z stderr F rsyslogd: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libuuid.so.1: no version information available (required by rsyslogd)\n2020-12-04T20:12:34.979165Z stdout F 2020-12-04T20:12:34,821679100+00:00 - nginx/run \n2020-12-04T20:12:34.9801996Z stdout F 2020-12-04T20:12:34,820335700+00:00 - gunicorn/run \n2020-12-04T20:12:35.0101422Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-04T20:12:35.0151571Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-04T20:12:35.0151571Z stderr F /usr/sbin/nginx: /azureml-

In [31]:

np.array(result)

array('Must pass 2-d input', dtype='<U19')

TODO: In the cell below, print the logs of the web service and delete the service

In [35]:
import requests

key1, Key2 = service.get_keys()
print(key1)

random_index = np.random.randint(0,len(x_test)-1)

input_data = "{\"data\": [" +str(list(x_test[random_index])) + "]}"

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(service.scoring_uri,input_data,headers = headers)

print("POST to url", service.scoring_uri)

print("label",y_test[random_index])
print("prediction:",resp.text)

ILCvpe6zg45EHwQDp6aihBhk7cz3d0DG


KeyError: 520

In [ ]:
service.delete()
